In [14]:
import spotipy
import spotipy.oauth2 as oauth2
import pandas as pd

credentials = oauth2.SpotifyClientCredentials(
        client_id='bc98f63798b24333bfe620e8b18f99e2',
        client_secret='77491b6e4d4d47cfa4fc70d3686d5ee4')

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

In [2]:
measure_features = ['danceability', 'energy', 'key', 'loudness', 
                    'mode', 'speechiness', 'acousticness', 'instrumentalness', 
                    'liveness', 'valence', 'tempo', 'time_signature']

genres = pd.read_csv('all_genres.csv')
genres.head()

,Unnamed: 0,GENRE,SIM_GENRES,SIM_WEIGHTS,OPP_GENRES,OPP_WEIGHTS,REL_ARTISTS,ARTIST_WEIGHTS,SPOTIFY_URL
0,0,backgroundmusic,"['reiki', 'calminginstrumental', 'yoga', 'back...","['103', '120', '101', '240', '102', '125', '11...","['mahraganat', 'divahouse', 'oldschoolukhiphop...","['160', '100', '121', '110', '135', '136', '12...","['NoemiNucci', 'JuditheAbelsen', 'FabienneWill...","['113', '117', '106', '131', '109', '115', '11...",https://open.spotify.com/user/thesoundsofspoti...
1,1,classicmoroccanpop,"['bluesrock', 'andalusianclassical', 'chaabima...","['103', '107', '109', '100', '102', '103', '10...","['shiverpop', 'gravewave', 'tassieindie', 'est...","['112', '112', '105', '102', '124', '100', '16...","['HajHusseinToulali', 'AbdelhadiBelkheyat', 'H...","['102', '103', '103', '108', '108', '102', '10...",https://open.spotify.com/user/thesoundsofspoti...
2,2,orthodoxchant,"['universitychoir', 'polishchoir', 'ukrainianc...","['102', '105', '107', '104', '106', '240', '10...","['sambass', 'funkybreaks', 'hardbass', 'hardco...","['107', '110', '158', '100', '119', '111', '10...","['LegeArtis', 'PeterMichaelides', 'CapellaGreg...","['114', '100', '160', '127', '115', '115', '10...",https://open.spotify.com/user/thesoundsofspoti...
3,3,tar,"['oud', 'persiantraditional', 'bansuri', 'tar'...","['106', '120', '101', '240', '103', '113', '10...","['happyhardcore', 'aggrotech', 'nightcore', 'b...","['109', '107', '127', '114', '115', '120', '12...","['KiyarashSaket', 'YusefForutan', 'HamidMoteba...","['100', '100', '111', '100', '100', '100', '10...",https://open.spotify.com/user/thesoundsofspoti...
4,4,reggae,"['dancehall', 'brazilianreggae', 'skarevival',...","['109', '105', '114', '101', '100', '127', '11...","['brazilianlo-firock', 'martialindustrial', 'p...","['106', '107', '155', '121', '160', '108', '14...","['TheItals', 'MykalRose', 'JohnnieClark', 'Asw...","['101', '102', '100', '105', '101', '105', '10...",https://open.spotify.com/user/thesoundsofspoti...


In [3]:
genres['PLAY_ID'] = genres['SPOTIFY_URL'].map(lambda s: s.split('playlist')[1][1:])

In [4]:
base = 'https://api.spotify.com/v1/'

In [5]:
def get_tids(play_id):
    url = base+'playlists/'+play_id+'/tracks'
    playlist_tracks = spotify._get(url = url)
    tids = []
    for track in playlist_tracks['items']:
        tids.append(track['track']['id'])
    return tids


In [6]:
def get_avg_features(df):
    rows = []
    dicts = []
    for playlist in df['PLAY_ID'].values:
        print(playlist, len(rows))
        try:
            tids = get_tids(playlist)
            url = base+'audio-features/?ids=' + ','.join(tids)
            audio = spotify._get(url = url)
            df = pd.DataFrame(audio['audio_features'])
            df = df[measure_features]
            feature_means = df.describe().loc['mean']
        except:
            feature_means = pd.Series({k:-1 for k in measure_features})
        dicts.append(feature_means.to_dict()) 
        feature_means.name = playlist
        rows.append(feature_means)
        if len(rows)%100 == 0:
            print('Covered {} rows'.format(len(rows)))
    return rows,dicts

In [117]:
test = genres.head(20)
rows, dicts = get_avg_features(test)
test['PLAYLIST_FEATURES'] = dicts
avg_features = pd.DataFrame(rows)
avg_features

2JsqQdMbBC2hh7GjsEjKFg 0
2SoQc6bippmMyUTn1qditv 1
1KZMCWImik3vA17zVLJ3c3 2
1i9zTcQqWUBTgIuCgxDW3S 3
0TcXdt4sbITbwCwwFbKYyd 4
73BLB2ZMlXTieOIXEeOVDY 5
771SM1pTZn9UnCESjwHmmE 6
3sEVkAftmt4GT4hheDpDsC 7
6VBe6cQvmjbByAhWav0Qbq 8
4CVeMaTHVqnDs13oYp0Hs6 9
0wJXJueXiwCAA8EYPYZqOF 10
3kbzqWhOrhZFXME9PpL8dg 11
0HphLqHgdLI5FsTHX3wCj3 12
2PuF72PN1DvyJOZJSk5L9I 13
52NjfVJ2mAEaeOorFvrGvk 14
1KO29VXKVt1zMzch57JlRU 15
2TX3gfOXHHoHZiMLK0SVmq 16
1JEkVeDauPlvYRDE6sOcGJ 17
1ulrJ0uK6zy2P4bruCmiZ4 18
04ZPZ3SUOh8GvHakbwkkg1 19


/usr/local/Cellar/python/3.7.5/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
2JsqQdMbBC2hh7GjsEjKFg,0.227639,0.074243,4.520000,-25.721200,0.670000,0.041912,0.920990,0.898570,0.119072,0.096035,91.654870,3.580000
2SoQc6bippmMyUTn1qditv,0.527290,0.647120,4.130000,-9.225800,0.720000,0.083125,0.467653,0.066714,0.284203,0.699050,120.122950,3.560000
1KZMCWImik3vA17zVLJ3c3,0.213437,0.161839,5.450000,-19.102940,0.560000,0.039790,0.969230,0.216151,0.146542,0.139981,102.162150,3.710000
1i9zTcQqWUBTgIuCgxDW3S,0.440590,0.348267,5.130000,-12.713870,0.710000,0.055350,0.899800,0.856770,0.142818,0.397623,108.124500,3.620000
0TcXdt4sbITbwCwwFbKYyd,0.789700,0.518490,5.200000,-9.983860,0.530000,0.134207,0.154024,0.112642,0.149378,0.736640,118.716180,4.000000
73BLB2ZMlXTieOIXEeOVDY,0.776800,0.638372,5.110000,-8.659200,0.500000,0.092803,0.079987,0.318375,0.218062,0.677210,132.763550,3.990000
771SM1pTZn9UnCESjwHmmE,0.397250,0.658711,5.690000,-8.133780,0.460000,0.042152,0.091234,0.858610,0.182863,0.145135,124.764590,3.810000
3sEVkAftmt4GT4hheDpDsC,0.506040,0.406400,5.340000,-10.799600,0.420000,0.032828,0.521340,0.006089,0.167645,0.343213,120.341080,3.760000
6VBe6cQvmjbByAhWav0Qbq,0.686570,0.847820,5.520000,-5.284760,0.950000,0.054894,0.249455,0.004727,0.294234,0.855240,134.624780,3.950000
4CVeMaTHVqnDs13oYp0Hs6,0.369452,0.867490,5.460000,-5.844600,0.510000,0.084674,0.041557,0.479548,0.229207,0.227671,126.763170,3.860000


In [7]:
len(genres)

3652

In [8]:
chunk1 = genres.head(1000)
chunk2 = genres.head(2000).tail(1000)
chunk3 = genres.head(3000).tail(1000)
chunk4 = genres.tail(652)

In [9]:
rows, dicts = get_avg_features(chunk1)

2JsqQdMbBC2hh7GjsEjKFg 0
2SoQc6bippmMyUTn1qditv 1
1KZMCWImik3vA17zVLJ3c3 2
1i9zTcQqWUBTgIuCgxDW3S 3
0TcXdt4sbITbwCwwFbKYyd 4
73BLB2ZMlXTieOIXEeOVDY 5
771SM1pTZn9UnCESjwHmmE 6
3sEVkAftmt4GT4hheDpDsC 7
6VBe6cQvmjbByAhWav0Qbq 8
4CVeMaTHVqnDs13oYp0Hs6 9
0wJXJueXiwCAA8EYPYZqOF 10
3kbzqWhOrhZFXME9PpL8dg 11
0HphLqHgdLI5FsTHX3wCj3 12
2PuF72PN1DvyJOZJSk5L9I 13
52NjfVJ2mAEaeOorFvrGvk 14
1KO29VXKVt1zMzch57JlRU 15
2TX3gfOXHHoHZiMLK0SVmq 16
1JEkVeDauPlvYRDE6sOcGJ 17
1ulrJ0uK6zy2P4bruCmiZ4 18
04ZPZ3SUOh8GvHakbwkkg1 19
6B1R2NhaSBVDEuKRDqhjAo 20
6oJ7aTA1gwk4WaPTB4l5oo 21
4K8cqoXccxo7fPmBEqVKQm 22
27smuYGRZZl9q0BCwda8en 23
3RbRDy3iktE2Zf0AP6cCK3 24
4xuMjOPD5nNuJqxDLtvAo7 25
1F4wNx9h2luKTSc1Tvx6fv 26
2jqAkE36tKTG48IdPF1917 27
71kedeh6L7P0equUL0vOqe 28
5HqaBtG1Clq1g28O0wKOAy 29
6tPZbh64wYuxEKSynffYk2 30
2n39wrdsYsdsEUrzMRYVdo 31
4P9bwV4vyjup4LWUj2iMUc 32
6KCEBhb5eYhZfJQQBL6W3Q 33
39v3x0pnnnKizjsCpvwd0p 34
4FlcRYTkhAqOWkQi9C4dss 35
2v7uYvX2h5hNSukaqp9LCb 36
5oqAgQxfCjjD5ejU76a8V3 37
5kDzf9eWk24U16EBkTDdpb

428kqH5lzImD4NkPLoZSrv 306
3eoGAwNztSSx7Pi6dC6GFn 307
1OXxQpaCXYbPe3lYjUdcuq 308
3y7GFNHSVQoII6mOYTvJaO 309
07dKI0FazIH66pe9lpMKYA 310
1gId0ogv6JOy3be2t736DE 311
0kOPCb8wKgaGA6kN3o6klO 312
2pSeH1YQCSICs2knjs7e5o 313
6Hhf2afEqaDe0Vut2AtlYS 314
5HQuEkc6b8niI4AdsQMrh3 315
53Rj4icQg1wx38THnNcMA8 316
2c8ynEyyWccPS7FTsB3dEC 317
04ugFARyXFS9iQaiHjes6i 318
7mXRIC5bSB6ZWOEPNfP0hl 319
009ddWL6yPxpgfLY76Z6x2 320
0jniswZ8mF0KAGP07FKFel 321
17J6qYVEkvx42RNjjqwWLo 322
0cRVSi1eXiAYipZDgVU11c 323
1qLMlVunYKqw285SAjZEzJ 324
4Jw47hdV6OqSfXh5z0HvZJ 325
4c8FDUUCm33WYC9xztAGYT 326
22dHFD1MN2DKvlSCDUcnDl 327
0CaVoV0ZC3aUxNSb8SoZVf 328
2sgKS2XmIrYZo9K94cfVOi 329
2lZNE4nnjvCXWnMHa5kc8o 330
3Xf5DQq60Gssc5ccah89wW 331
5Wa6334bix3KvCzwThgNYR 332
4EOv49pXM5JJ7BTaas6P9d 333
0jRf2cuzt0bLNhZZsLFiGg 334
2bV5j3UH5k9rMxJywzTYKu 335
1eyCjFAnmJxBnrXvLndyOE 336
3j1YYS7Palpfb92qd9x7vl 337
535kYAMV6pRqLgCnOEqB08 338
1pCKc8Kc87BG4FdIGKaQ5v 339
6bOsuV6PvVwM3dzA7AKTtx 340
7AnZjCk6I1Nf19w2Igpj4H 341
0xZkDM0jIfrFATSeaGt9Ms 342
3

0M2STcvX62sBK5YuLTE076 608
5j3yYNLrqwPHmXzckFjOwW 609
4g8jY2NYAlj7sBdFHtrgdF 610
2hER2uXioqqYyU6CwjsQHU 611
6odC8x54AS8rzUqHupqLDS 612
0XsYSjgy8rwnIKalWCm9d2 613
3C24egLjqcCmIMgfztqI2r 614
3RnjdCOpB2uSOQlHin0YZr 615
6TGOKSobwpEoOtx80edFyV 616
1LPg7FVcDkLiDiEP3YcClc 617
40uOIq5lmbAApQTlleiVp0 618
0cuaFpexE9V0zr3s7l6yjR 619
58UKNHdZKjKa9q3gPPwSsC 620
7EIYnaUoJnYUIAvn2jzbAW 621
0QY6jIObQ6cOcRr30d0XT4 622
68qffdP3LzsrDkrmrt0xKi 623
21vmnDNKEQfYgZvtOBEpf2 624
0kDDReaApSTI0wI0kPOqhT 625
2FTKXAfnRZlKfOSPQnvc2O 626
7iXfcP5kZ4EPqSSmySAUtf 627
3urA7iwGhdJZDHWic62XbG 628
3F82ezKwcNctesxcgvlL5x 629
1Ph8pxls7GNLTFH7fcWxKh 630
2a4AkEFVA2nSuddrYs5Xfr 631
3bJfKd60ASF6rRzlcTzF8T 632
1DIyRhN8JzZ1Y54dAJp8HC 633
63rahDKdDo5xjhy0DsIafA 634
1tkjrVDJx0MwmNsoV5lgPr 635
6yZn81v1jP6Phi9S0tm4At 636
5Q3wLZoh8pyDIboX6XolLp 637
2snnzc6loGGiQ6gitdUggi 638
1fymlByuu9HabrD0hty0td 639
46zaMGzDdLfisrtepPsZsO 640
2NweJbjBo0jbtRgYOalUDO 641
7JEBOiPat4SDUDjcGgc4YP 642
460OzdtRygFt2sDZYDIlxC 643
5PU59fME6qieTTZZkomRbr 644
4

7ajQKzORUThQQYgmGOhIHK 910
3vjyDo80PnOOIAjaRLgQ41 911
196T2qP29EjWxI1x6oSiXa 912
4ML7JSFmkkJipKldJqZGbN 913
7lqV7lzog3hotsBkAVUJIl 914
1vc3RFXIHHqgy1hp2aL5EK 915
3VKGShOBz2FYrORLpgwTRe 916
2HibxdzVLGDPR5GIrm4cza 917
1ng2topoxV6TEuDgm2Ai90 918
5HyQ5GsKMRx7eTPoZyD7pu 919
6Mz4j5CTDBR3V369aRhLvw 920
5O5sX3NAyXbfZgjjEHIeTy 921
1uny7o4LNoe7C5qJJ8S1k1 922
1mvrm0uKYoDivIpWYMQWLH 923
6GRRlIuSXWKDB2uVxCTLaT 924
7JncFH6GAjyAobbWbIF2Qh 925
6MjdCkcY5tUfadaakBRINy 926
0pyHVjbjiyuutRjtP3hvNY 927
3TSPcCQfHfv9ga1UTKeBTH 928
46OzYiu5J1KvNDAwyY126g 929
7m4SvnJOObW34xVsLdXx30 930
74DjsGDWyJRTJSLGLX4cAp 931
4BXMH7TLydNEXR5awGJ65g 932
3rGaSnZjjAQsIqjOApHTQf 933
0VgyqskAG7vnQto157aNWc 934
3ihgsFVkN3qG7KnEXoJMHf 935
0VSDcgTOgN08cWVEkk4QBx 936
2QecrVF3lRAI082RQiH5pd 937
37qBQkNmniKSzKQSaF1gOw 938
7dJlO6VYlCSlfxued1pqso 939
6KAn1uGwrupBX9KI6jOT5N 940
2GA6waoTYbDuJM8c066jXu 941
3AQTwSrkyEbZBguO5TbFlV 942
2pxOA8NLXwZaCk09pjaerb 943
4iYWdcJhjLHD9iHCXqnv7z 944
6nAOWmyYPXdqQtAJ8AdpwX 945
0GO4HceKLkcQHrMkSvy9tM 946
4

In [11]:
rows2, dicts2 = get_avg_features(chunk2)

4KxHxwmVq3Q7Gg9wkyOwzV 0
623xipOO67LZgslyiF58Mt 1
5GtYXXydQZxdFlkIIvCHBW 2
151LvuVlFGqjjS4uQKHd4T 3
3yO7yAkgfetYQ2hpYb2AJX 4
3SvVSF8Ex8YyCjjaW9bojd 5
5S4nocceN9d2bFLsBlD3Pt 6
2KOiyljP0uPAaPSjripotZ 7
5vpvuv9xaddqd00dzpxTzg 8
53x6LAhXxhWxhV2TAPgw4u 9
08AHYrQ9Og6ZuhJtEFbtyv 10
1IlJGpU4HqNpfuJ0E879Dm 11
4cavePoqp7tBZ8FYfZaDlf 12
0u30dhCaWTOJQ8pH1uzDkZ 13
4L18ziUY8P9SHL0dv0SpLX 14
7BKZEIdyav6sz9B628eXSx 15
40auwv5lX0xeL2w7Kr6BoT 16
72YNCrIywbtPaFgFSQNI6X 17
3oWlrpGzB8SAslBk22W9lP 18
6nMGFNdf9QNLJ8xz8NdZp6 19
7ratOul7KberTWrn86l4O7 20
4TUWhdsP6yM5qB4ogDgXpJ 21
2Uu1r2ra4Xjiweb2VjDkjl 22
7KmV0LJiGPgoQFKAOqgWxs 23
1rpCUMoi2C2YUe67ZPSdC4 24
5gQyJA5uWrGtqblgCuuQn8 25
2gmlwXEHmmvxTG6JZN5uBQ 26
3H2s2HGReZFS8HKHNutpXg 27
50kZecUV5pY2SuJv3Ie0vy 28
2RcLS96OmD4fLXofGRlGTf 29
5ELJuzyWxmJSF24NtjZ5oH 30
49BlLjNkElJlX41Di10qql 31
50gbwtczrJEW7WRaI9077I 32
1BwLE6QTlmYAtosuvJcgY7 33
1thKr5e2Vm58eAyvPU6Uoh 34
4NCTzrcXK5FNg1mCOTvk8c 35
7uiDWyPL9wjqAeu0xizq1f 36
2nl7TE1YytDEEfs4Ul6ALj 37
6vx1kgPpIzxF492o7ShEvf

35TkC5o2mnTjREHgS8coYY 306
4byNu3ZWic9R7OgfT8jBFM 307
2gCb5sPRAaufQVrfBOLgdN 308
18b0DBiqVtRuj34OKPCgj6 309
3bjbpsbl2mzaqFJcyQNGjE 310
5TDA5AEIBeLtwA6zpETBe9 311
1xdLdgcB08v8hlQt8qRbU1 312
0grTiL0EW85nP5udOaUBxl 313
6uL4zN0tryTmOsrhcknaIS 314
53scIdMPn5Px3PdVOY1nC1 315
44UhH06dKGdI3cqbst5jJ4 316
4LMXDmIn72kgOz2tlneVG2 317
0IO0gpqCNRyp1tawpP9YSN 318
1v75VUwKR28oOd6Lf3C6JD 319
3FRB6984xpSb7nal9CtWH9 320
7d4A45WwlxWLN2E0LIrUxE 321
5PUxbB9Jk47DOKdYKMoYQs 322
2nuFDRMV9ePZJba5V94AJ8 323
2ZrBy8XJhh2lEyXK1jXJ6n 324
6pt4hCDsGV4O1xEBGhaiLI 325
4oBAlPMMM8HqTd9ee6GiRr 326
3CdYZdrHIJcTGK0ooV3ygU 327
1gyxfWb5nNDYEOC7oPN2Qk 328
3pDxuMpz94eDs7WFqudTbZ 329
0IwWYZCm6MJHuzknIZHw7B 330
2NMVEGeiUeIosCna7tcMUp 331
4SIr7n8YCW0B6cKMdc3x4h 332
4BNdtoAweEyMunEAPyH754 333
5YR7tBihIt7vVZPrSGmQYt 334
293CDKWmDYutJjtVGJaq3L 335
3rGqaAFnWBXfSlzQpiQAQh 336
6iwoiGbEuqmejpaGrxJWiT 337
02lAxUTKaWXNEtYAIEaZpP 338
4v2etpAUyEX5sRicWn1IHC 339
0F7i7LpeujWblHYMdonb0g 340
2fwbem3okAolbotKZaFsk2 341
5vZJbTAT5UlOVc0jhWT3ao 342
2

0hkXwJmcwu7hABTyspKI9b 608
6auaeyb8nkpRdYdJ9UFxcO 609
7JMWNzG2Av3PkWgZmUO5x9 610
0IyFLwxEtAB5jszehTpWFJ 611
70f3S3d60uyOdssfCZ9AEe 612
6hlHOKDB7Jxpz2cg7Jevuq 613
3gMqSTa0UJO6c8CQDtQ2OL 614
4yHRpNzpFRWeKS0WinqHtW 615
4LR7zJ6HkhAJFhQm4eTfqS 616
3smhgA5tMEWCfjqB6WF54r 617
51ZET8D7C1TIlpbFsREleh 618
62KNLpt0asiJcAw8hihPJq 619
1FYtDs08idk3VjvtgKRjZ7 620
0X8BD5Y2hyyC6Vaccx0AoF 621
14bpCLkVLcFmPMzGVXI7tN 622
1w8TCr2tpXhXXHLmYS5gkv 623
3NdAzHVxjhUdezXDcT0K8m 624
2X35a4BVWpyP9BwcyRRhJD 625
2mb1CA8bzqttQcD1HJaPyn 626
5jaGOpLsE7tSbE060h8U6C 627
1jraqJZXXsiCwPb9Ykobhv 628
1DtpofkGb09zLXCzsmlQ8C 629
532PtpsJMVHeWcE0XOvUKp 630
2dVHwR9NRScJP0Ss4G7t0B 631
1YzrgJGO5xdUd6oMPcpjqq 632
5scILYXkjOSPXJVC494rCD 633
4ivlJRUHZWiP25Fm2qN7Dv 634
7lCazMg1AalQrclLnSpn7v 635
0ZVSWcJIf7cvycEn9HUvps 636
7hNOYSiftr0KFWblJ6ccTi 637
534dDLa6qUAXMp5paJfNbu 638
2vk7YBjoVhFTgGBxlKcRPF 639
4CfyHGpt8qPlLXlstcE2Wh 640
5xxIhwNhCZlkyFKpA8hxRj 641
4EYSGTuqe9cVfSVpX4gtGv 642
6PtT6ZnKDSmeaczTMZV7dl 643
4btmxVWpjsCzIN2nYQ9Bcc 644
1

7Ew6UU6dhWHNs8f7qr9eA5 910
1ZoIbk8whEaIhk5q1UQYre 911
06Bclwc54xNjzibZZKl6KX 912
2eRw9eqaDyvtlYq1y2RlxR 913
0RJFKia2RsWkIk6wseqOoN 914
20K7P3XPAqGdC0P42EAY1j 915
18DLKGkmiAelCa1fE0WhCp 916
5l5jOUgJm8gedEcYijsBcK 917
1nDSNgH9o8lPKZyRC9tAwR 918
45GY1sw2pj0LitunAGcMz6 919
3GL4HNVOg9IqD5jxVpSRp7 920
6tLZeHeJuat5gr4ygTW4Fc 921
4SraIxQeuuXNWKokMAZvhl 922
212BIjWzEJVNkScamNoETN 923
0TnfKtQvtS8fcWUgXXBj8w 924
0sCxbu4WgloqSoxM5WZhjd 925
3UQbqTKIxehTFCdFyjEQUn 926
2lvWEEJ981sRN5GQzBvayl 927
7DrJ92Lc9UaVB1rKM2UGsg 928
4Xw2kcuC3YkvtBYlhlWSsE 929
3ld6kktsIosfOrMt3AVPiU 930
7wQsJI14eRUiBLMk65F8DS 931
7rAlDSQkUjL5aaGe5PQDd1 932
24nFbemYXcm4Ep56SCHQdR 933
7trkrHnBsGGGaQm3KJgVKP 934
6Ejh1mnEuKQS1WiklGSRZd 935
7ptDZTQFdaqA519GUfff8k 936
24lpzMRS776gtph07QO0AK 937
4KLH6GGDeD0FFxjlz2ifeJ 938
1fzcR3U9cUpJm92J14ThFS 939
36twaQ7dgJjhqW3IEvfbTX 940
47v4HO6aShu1jyLNudBinv 941
7y06Quhxd73Bu5l6GNJDYw 942
0T2aOQSexU1FB4aQQBSdQd 943
5j9E5msjnYEL2PZLjRzfQy 944
4ZInSdk8VMhCM6WmRmfy9w 945
4PRTrvutXqjv9Ec5CVO7w6 946
4

In [13]:
rows3, dicts3 = get_avg_features(chunk3)

0VnmxIGhsByRdLmRtRaOus 0
4pgTiBCem4FrZlZwolDKNI 1
3xEJhFDLGO8n5MYNE8vjjg 2
4LfTA71lypx7XfDpvMZXae 3
5laz28pNMW5qnvxtOGDjgP 4
7pBdTAU2WIt4hx9jL9LKqo 5
6QE53B7NfNuEJ8ZlwJMEA3 6
5x4b61pvg45R2O5LHFBjOH 7
4t5PSWEsLQeZ4i90PEpW9i 8
1hDdmYxsuVfuyVGKz1mkcI 9
6lxiTORjbiO6lYSldqNSt4 10
19TTQtvqRkqHiIwxdHCbhS 11
6TFuQZNFJiSZZILqBd1LnJ 12
2HAQJtMJXSdgthjyE9TZOI 13
1QePwfTwgTHDxNTfycMOqE 14
1rYUoCN3Vc5KMfqsKvlVtv 15
4bMTSiI4AC7Mp0TCOMis7A 16
2VkWOPUqwKkeBWMqcFx0L6 17
5VnHdgsAhzWNkd29Rv9ErH 18
3Nowur2XueyjlPEFZ2oQVA 19
7eF49eqkrZG5wFtihgZePd 20
5YlKVBQiPIlV4uxyQVNmWm 21
3s3zindCeXj6wbEpjlP6xX 22
1RWc3d424qgjr8HTwCVh6A 23
4WGe8j09ND0yhNGN2LRuD7 24
7jAbh7lCrzQWvRguJ3cjWn 25
4o7TCoOT3h3CBBV8d4mc9s 26
60VqjFK8tIzmpc5Q1NTosK 27
4UUXB0Mz6E3VbmRdbi2o5P 28
2PJuWfVaCmG9Bu3OlHFzfy 29
6AsYWsnau4HpWHzxyNoA1a 30
4HozwtuUKoDxCR0GrX7o8L 31
4ILgcuCSyGWNY1RejVHgiN 32
0qxI6lRUwws4qXyriYiINB 33
5bywGHY4NQKcTmGXAVsf3N 34
2TJHOlrTUoLMHs9CTyxN9n 35
5HintVVklsFbeTTlQFqmlw 36
7iznfxFejfxYQgIjKo5Xyt 37
2ID0zU2K42h2pRR6yZqmDB

7z91neKtQAc0DnOyyHnr67 306
4xVyoTeHmAQ0yqz3D6GJx6 307
3ShagP0yJAuPN8psJvAz2n 308
3WESokAhiJymMBjplSo0LK 309
6tJl39oFvuNfQdoM0FloAN 310
6C7kz6eR1Sf12ZBrzl7aD8 311
2owRoUVlHqTdJF3FmF5uwZ 312
1x5KJwBNPIy3bYltlNaNDK 313
59qMoqvlNat6Knc0feSWJn 314
5RmdN1TuYgoPmnQmMYE9X6 315
6BIS9zOW7eMTURxORLJi2g 316
34Co5PeMuUQjCqP8s2c8sZ 317
6NtolKQzqsuJxnDi5NgBkH 318
7oDabKiKmedjlCG29ZHZsg 319
1bx8VTvb9Thu8MWpnhuP6k 320
28m57W4UI4eetkNu9SvtX9 321
2CTX0IxBoq69Kq4trHCzAD 322
24GT3KBxuEkuI6ATbiNBtg 323
6rhKHVL7MpASDhCtuY3ISy 324
3Qb1SuXPEkMxqQB81haHqs 325
6YlhnR407LgZfCOOGHZfiv 326
0tb4mctI0W8DgwlQzPx2Hd 327
7g97O2qHKYa3VZxOKMmyiL 328
60NgGlktCblheYPBSSUUXz 329
7BLKSzIiVrkhVe0tQFTtJg 330
3mg8CrTTMG66HCzZroE6TD 331
3SbmBdyPiGj7fBc36n7BxE 332
1ymgn7ot4hxSvd9Qrc96HY 333
1SPR7yk4kwmz09NovuUzKN 334
2DEoTCwNKhPrscHszesnoC 335
2d32hJkHO6816u3Iho8NMC 336
4nXjDD7cf9Ljrcjxo8iiir 337
1MfuuCRndwqJIGz1yW8aHH 338
5lw0UguN3MakCpCgkxFGpM 339
7615WtgKBqapPBpBB9lxUW 340
51YToRmnFC3U4iPj6LgHgF 341
4XhnpVMVwnL8e7EHxA2Rna 342
6

68kxnt4QHyJ4cLONsi5VsX 608
5eXtMliOlZWHMcDDRkANhL 609
571OWCRxZMCGVL4Ste9FVw 610
7yYi9aJirMqgnn5XlqOkZp 611
6Z9H0dv5ldWEmqk8lUTvqV 612
0VIFRdDKyTqPghiYUu191o 613
1y6yh0oYnU38wZsGARkcnX 614
6nOAxBntpTFNYIxO8DUEM4 615
3htJqjjCUIndBa8q3ngCUL 616
3YjCK5F0D78d0WXHcwQ7tq 617
6dHwd8PSz0phwZkwBRLMm1 618
3SKPT4J5y03Qn0u4iCtish 619
5c8RLU4cABQz4E0AWFc22P 620
63PEOiPwgBRps6iW2emDAl 621
5VedI2ZC26yEKJ8JTA7OuW 622
6WerM05v6Nw7rAFpjWRk7C 623
7vyHBfRfPOT8Sxy87P5osJ 624
35Ezi6bCni1zwVOW2rOWDj 625
0YLPrzWklBgy152fwVL0iT 626
2fv5wnCHoqJtxmaq9IAsiV 627
16abo0oaXgiL8a1NU9ZPWQ 628
5Wh14KMrdMwRzU4lU8Zc9z 629
4zC9gA0aRcK5KP0hANQmFy 630
6THzRYh7cRGUHAxQjkqKT8 631
3KsDNETNZ38sktwgKwlkWB 632
4PhVCYY5V1hdjkaL7LWcvE 633
2CUoWOcTZtUfKD1DeRFFcK 634
6f7acaGPASiQ3TAEh6oLEU 635
1HDK9LHAXjMQEbEhh9Lbfa 636
7uGtuTM65i6r8zxYuD7Kwp 637
0bHOelJ0VL9QgC3c1UIoJM 638
1uFmFdP4khly2qaImrnOz0 639
6LJOGOAWRYLxDPGirVxTO3 640
5eXcXox905vgCOp8bE4TJI 641
5qQLE1iuD3CKOEXIb0fi7Z 642
1gGPDlhf1YsRNJhkQE7c4e 643
7xVNaEai3Tk9uOZ2p3sdg5 644
4

5M9OhWILZhUHrJZapQ7tSO 910
33jX0HQ4Ghh1RrEfZqFs1O 911
0yaAn9ELCJ7WzLhYJ3DHNi 912
1dHm6fhlORhY2O1v6aO4fc 913
65cw6LKvXQFFHR2iwNjZHY 914
5ls1UJ1dPxJh3t0W7C8m6R 915
2cXDt1ed25OJuZkDpEXAYp 916
4cR1c9NaieHXwFyHA6LlpR 917
6uSkf3ARKkBhgf8bQE8NSN 918
4scdFEeqzm7Z5n0cJghVFD 919
4ysJLFPAYAt4Uqyzo7OasJ 920
1jVq5hfumPrlKudzlaQ0Iy 921
2tOnLiRR4UNjLEewu0OEYC 922
53c317ocGcanb04Adxe8Es 923
7pG25ZvJoouVIKkbGoA4kT 924
7vtKvrXpzwYH31cDsCSL5z 925
7Ky7mnFOobBYJMFnNtLN8O 926
5kdPHafCZx70YvDHkihSuT 927
07K0fnv9a9jclwEojIFMha 928
6Vmrzb8WLxoZzUX7pQ5SRX 929
2qKl9xYWzXTM65TG3PFmMc 930
3ajp7PXuC2xPYxCuetjcRH 931
66k6nBXFo6of5ckPWCuCSi 932
6S5gl15J9q0DHwi8curRUi 933
4SbkCOtqMhatjtiEBf0fOW 934
2BsP4wBGZt4mpY435CfQGR 935
6rKNApKAQaPnemPiI8dIJj 936
6UfbwvRitXs5YLmPg8prFL 937
2b1a66G3u1GA6SrCYrR70S 938
2V5cDE76iiL5Gm8vdbHePQ 939
5TCT3htdio4Hik2pF12ky8 940
47quI8Re3ItWH8Rvn9HSvk 941
2cmBKubFYRJ8QfvqJibiTS 942
5kredHbbKjb6EUN5exsaQh 943
2H9lhkryk077kmxq8UOYkl 944
3iL6DO0YxEvjK2q0JYzzvH 945
6CBOBJn4M4ql1LHSdHhkbz 946
0

In [15]:
rows4, dicts4 = get_avg_features(chunk4)

7N35r1mrsCLe66LczBV7EQ 0
08m9824rPzzzSV8kUNqaJ7 1
5166vpEenpiy4kVU9XafbS 2
2TsSOQsGVFwKfOZLra0x4E 3
5UgGwJxLX6i8u422mj1xmy 4
38h45JzHtrOjAJAtbiwNJ3 5
082daAvMUmkVEgkolt3wCw 6
5uNencS97nBENMIyhtZguS 7
7u6WmGIdrl2LFd73vZhBJQ 8
6YVEV28McmK71mOrvNmVzZ 9
6bpwGHlRRLkVqfmOpqfmvx 10
3weJSiTAEBXFb2GGzQdzDH 11
2zxYQBTWiXgpQEyQggYSTm 12
3NfxAsLjZmCpQIlytSFiMB 13
6JqQli6WY1G043Mihy3MBp 14
1MThleZUevCoZ72rYIRgRJ 15
6xLA8mHVCxyHRW9Mxfn8Dy 16
7BKzCGpmIAZeheQzxf9ufa 17
1LWJWwk9c4RDbyhqXWySH4 18
5yQ0TDwTMx5PwL1suz5PC2 19
4ddNv006w0q43fibFKQVW6 20
1jfoRuFq7k7f5ZBEaninHm 21
7wx58jJTsUZs2EIzNfGHth 22
27nN5LFsfA2HkilFH9fRCy 23
4jr1tvvMa8bYtXGQZPFn3T 24
136hER4CtHABIFZjI4FTm5 25
7uVfHvphUCmZCSgbGiOCgw 26
6r9pKD3yNDbAm1iA0FhDM6 27
4HoT33OJOwvMElhB4xEeFs 28
61BVo2kYujeavfw38gf5rm 29
5otIcGrLz05cFq5Erhbdml 30
5sLhmwkeZsqxUfx1871zjC 31
7Mb1RVFEVwS1kOeMpbCWbR 32
0TFl3Tevi22JWs1KWTXcAo 33
7roO0fEOrwx0GdJlqB3gSH 34
06nltIE2BZRD4KtSZ0Scnl 35
7ouqnbBBJVTn2ekY2Us9rA 36
3FDQXSzsETVYzbrfOEEJnQ 37
48uCDGE2YYjZsPSyhZXGP5

5LESZghBqSzHNi5DR7f3A5 306
1u4LMI2YFyDRBtCJ00rbzm 307
4cJaiQ3OSsF2kxYbPJZkQc 308
6cFjgaI6dLy1cM0HTKQq8Z 309
4b6NJsrgYmVbDsOclVNDud 310
1KJB07h1cyZlw6I6sHAide 311
3c8pTOYECLUWMqLppQTc1F 312
1VT5zGqqy4CSw6zArZoZgF 313
1lMgl0D1HZVvtq16pB13ew 314
0GhyufHLiAxUoRloo1mZM1 315
4nSrizFStxySLAXtAkJlkd 316
4VfL2DMccZY34iNewTZKWZ 317
5bTmOsC15wxJWZozRrKqZG 318
7xrsyHt3RVeYJRSX5M4gzZ 319
1BsskTfK2Sfg5mJI1o4MGr 320
1ACo2WIQsolnvZj3UhHTuC 321
5KudWqpwRKdH5CY6yQ1nwA 322
5BJ5rG6hFaItN8dIfZukGM 323
4Pgl4JWg99RspoFbmSjq6P 324
4EIGwSoao5WkHd5YVP4CfG 325
0MgTVZhnxixIfszqQRnx6v 326
7o7kEPpjIe3yRRPbDFJSIR 327
6Hq2IbgsGDBMUNJP0ksfVs 328
4qpdGrwHTenL9t4m2nAbK5 329
1nJEcAFNHlbITIZ6vokLFi 330
1rL1br6e5ksgcTIkgRLEiF 331
2XKIPP83gL3CAONT0n5ZmT 332
5Eavu4IrTnwmZZQbAM18KO 333
1pJbr3AF7AZtbMZBmyvBRn 334
4nY6eWfRNKtTzG1oZLer4I 335
6aRR1ZmHR6ehjbhlsoctG1 336
0vWBxXexiRvWBnfTwswJQH 337
1D7jPq4bYfm26hI4qkol1A 338
38HIhRQM2qSIRTjUW00xwp 339
3yP7LJ0wT19xpeIJxJ7GJF 340
0BhYpMRb8Y8Qu76nu7gFmh 341
0Nar1vUd0plNmYHqPPznaf 342
6

2KR8vprLTSBgSCk2dI2wTj 608
3SwCXSJe5qaCRlxud8SDuU 609
0fe5Mzpe1pImBgYciFmiYo 610
3zE5pYwFbC6sSjMRjvhePx 611
02NPFyGAScohVgMt3JN00H 612
2lRquSJcjNvHMjlEl8zLbg 613
64KEXfUapxhkmiqKDsBrbN 614
5nMh3IcKI0h3bfeiwdoUrW 615
0mkpZNuKedVZrRUsJVLNar 616
6fvk63guE3C9CwuBHTnELe 617
0GqkGs9QSKFwUAku4VMj3v 618
79AOdu3ELUDHkDPJT1rXUt 619
1vFKPl9m36U5Yx4UAZFaAr 620
5akdeWIPyZlIYz23s861Fn 621
7Kg7DFSptkBEuol6EKr8f2 622
3VvwVEZaPYQfI8ZZbOCW5h 623
4vnhpAi5S0LSNQfzWSpkZs 624
1Cf8ZdtSkyItY3LKaiZnWD 625
36VbduhjU89A2hn4364aba 626
6BfYBqrSm30CXrqFwecv5d 627
1FcWxTM72potsWHiPQKmZC 628
4w3F7UV3j9j696HYl16IsU 629
0jFvYjzBeEgvfwJ0TSTNVm 630
6gS6GzxIiIADmPTa4xYvcC 631
0mKxo1d7Rshs6q4BrCEL5I 632
20wI9KYEcMyhlyPdVMJmMA 633
69OpOkQ8dkxZGn4If6ry6G 634
3JDDP1JjveG5kUAnEIQK1l 635
5d9dEMb9Ezh1HmFfBdSbU9 636
3jJdVfjC6emcsncxWd16Ud 637
11UvQwdCC7p4WoF9AyhIdq 638
2U7wEb4EEDbiAs3CCOoUfn 639
2himelMaFON7yDwQf9P8Kq 640
1Cr2fJmkdikZdux7SXq3Tv 641
5mJMwtywZacymdtdT21ABQ 642
6IywiOKAoppQqNyugHBSMW 643
6k7Tv57wY0l9lrPGhYfrw7 644
0

In [95]:
all_dicts = dicts + dicts2 + dicts3 + dicts4

In [96]:
all_rows = rows + rows2 + rows3 + rows4
df = pd.DataFrame(all_rows)
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
2JsqQdMbBC2hh7GjsEjKFg,0.54204,0.62055,5.5,-6.93119,0.87,0.04358,0.221045,0.011131,0.182668,0.459032,128.02859,3.9
2SoQc6bippmMyUTn1qditv,0.54204,0.62055,5.5,-6.93119,0.87,0.04358,0.221045,0.011131,0.182668,0.459032,128.02859,3.9
1KZMCWImik3vA17zVLJ3c3,0.54204,0.62055,5.5,-6.93119,0.87,0.04358,0.221045,0.011131,0.182668,0.459032,128.02859,3.9
1i9zTcQqWUBTgIuCgxDW3S,0.54204,0.62055,5.5,-6.93119,0.87,0.04358,0.221045,0.011131,0.182668,0.459032,128.02859,3.9
0TcXdt4sbITbwCwwFbKYyd,0.54204,0.62055,5.5,-6.93119,0.87,0.04358,0.221045,0.011131,0.182668,0.459032,128.02859,3.9
...,...,...,...,...,...,...,...,...,...,...,...,...
5kAulE1JAusA4SlfP0OjYG,0.54204,0.62055,5.5,-6.93119,0.87,0.04358,0.221045,0.011131,0.182668,0.459032,128.02859,3.9
42dGKCvDkq9aLYNzKwatSu,0.54204,0.62055,5.5,-6.93119,0.87,0.04358,0.221045,0.011131,0.182668,0.459032,128.02859,3.9
4hdX1gS6iDhWw0ll0VBBxU,0.54204,0.62055,5.5,-6.93119,0.87,0.04358,0.221045,0.011131,0.182668,0.459032,128.02859,3.9
69AS6y3vT3IRvzBiPwvqP9,0.54204,0.62055,5.5,-6.93119,0.87,0.04358,0.221045,0.011131,0.182668,0.459032,128.02859,3.9


In [97]:
df.to_csv('play_id_to_features.csv')